<a href="https://colab.research.google.com/github/dn717/hello-world/blob/master/NLP_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import matplotlib.pylab
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import rcParams
 


In [0]:
dataset = pd.read_csv('/content/drive/My Drive/housing.csv') 
dataset10=pd.read_csv('/content/drive/My Drive/housing.csv',nrows=10) 
print("Here are the first 10 rows of the dataset:" ) 
dataset.head(10)


In [0]:
dataset10.columns
for i in dataset10.columns.drop('median_house_value','ocean_proximity'):
    dataset10[i].plot()
    


plt.show()


In [0]:
dataset = dataset.dropna()
Y = dataset['median_house_value']
X = dataset.loc[:,'longitude':'median_income']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size= 0.3)
print("There are " + str(x_train.size) + " training entries and " +       str(x_test.size) + " testing entries!")


There are 114424 training entries and 49040 testing entries!


In [0]:
import torch
from torch.nn import Linear
from torch.optim import SGD
from torch.nn import L1Loss


In [0]:
x_train= x_train.to_numpy() 
y_train= y_train.to_numpy()
x_test= x_test.to_numpy() 
y_test= y_test.to_numpy()



In [0]:
import torch
from torch.nn import Conv1d
from torch.nn import MaxPool1d
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn.functional import relu
from torch.utils.data import DataLoader, TensorDataset


In [0]:
class CnnRegressor(torch.nn.Module): 
  def __init__(self, batch_size, inputs, outputs):
   super(CnnRegressor, self).__init__() 
   self.batch_size = batch_size 
   self.inputs = inputs 
   self.outputs = outputs
   self.input_layer = Conv1d(inputs, batch_size, 1)
   self.max_pooling_layer = MaxPool1d(1)
   self.conv_layer = Conv1d(batch_size, 128, 1)
   self.flatten_layer = Flatten()
   self.linear_layer = Linear(128, 64)
   self.output_layer = Linear(64, outputs)
  def feed(self, input):
   input = input.reshape((self.batch_size, self.inputs, 1))
   output = relu(self.input_layer(input))
   output = self.max_pooling_layer(output)
   output = relu(self.conv_layer(output))
   output = self.flatten_layer(output)
   output = self.linear_layer(output)
   output = self.output_layer(output) 
   return output




In [0]:
!pip install pytorch-ignite 
from ignite.contrib.metrics.regression.r2_score import R2Score

batch_size = 64
1116044_1dconv_reg = CnnRegressor(batch_size,X.shape[1], 1)
1116044_1dconv_reg.cuda()



In [0]:
def model_loss(1116044_1dconv_reg, dataset, train = False, optimizer = None): 
  performance = L1Loss()
  score_metric = R2Score()
  avg_loss = 0 
  avg_score = 0 
  count = 0 
  for input, output in iter(dataset): 
    predictions = 1116044_1dconv_reg.feed(input)
    loss = performance(predictions, output)
    score_metric.update([predictions, output]) 
    score = score_metric.compute()
    if(train): 
       optimizer.zero_grad()
       loss.backward()
       optimizer.step()
       avg_loss += loss.item() 
       avg_score += score 
       count += 1
  return avg_loss/count, avg_score/count

In [0]:
epochs = 10
optimizer = SGD(model.parameters(), lr=1e-5)
inputs = torch.from_numpy(x_train).cuda().float()
outputs = torch.from_numpy(y_train.reshape(y_train.shape[0], 1)).cuda().float()
tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)
for epoch in range(epochs): 
  avg_loss, avg_r2_score = model_loss(1116044_1dconv_reg, loader, train=True, optimizer=optimizer)
  print("Epoch " + str(epoch + 1) + ":\n\tLoss = " + str(avg_loss) + "\n\tR^2 Score = " + str(avg_r2_score))

In [0]:

inputs = torch.from_numpy(x_test).cuda().float() 
outputs = torch.from_numpy(y_test.reshape(y_test.shape[0], 1)).cuda().float()
tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)
avg_loss, avg_r2_score = model_loss(1116044_1dconv_reg, loader) 
print("The model's L1 loss is: " + str(avg_loss)) 
print("The model's R^2 score is: " + str(avg_r2_score))
